In [22]:
# +--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+
# |ID      |n_adults|n_children|weekend_nights|week_nights|meal_plan   |car_parking_space|room_type  |lead_time|year|month|date|market_segment|repeated_guest|previous_cancellations|previous_bookings_not_canceled|avg_room_price|special_requests|status      |
# +--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+
# |INN00001|2       |0         |1             |2          |Meal Plan 1 |0                |Room_Type 1|224      |2017|10   |2   |Offline       |0             |0                     |0                             |65            |0               |Not_Canceled|
# |INN00002|2       |0         |2             |3          |Not Selected|0                |Room_Type 1|5        |2018|11   |6   |Online        |0             |0                     |0                             |106.68        |1               |Not_Canceled|
# |INN00003|1       |0         |2             |1          |Meal Plan 1 |0                |Room_Type 1|1        |2018|2    |28  |Online        |0             |0                     |0                             |60            |0               |Canceled    |
# |INN00004|2       |0         |0             |2          |Meal Plan 1 |0                |Room_Type 1|211      |2018|5    |20  |Online        |0             |0                     |0                             |100           |0               |Canceled    |
# |INN00005|2       |0         |1             |1          |Not Selected|0                |Room_Type 1|48       |2018|4    |11  |Online        |0             |0                     |0                             |94.5          |0               |Canceled    |
# +--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum, count, when, round, lit
from pyspark.sql.functions import to_date, concat_ws, expr, explode, coalesce, date_format
from pyspark.sql.types import IntegerType, DoubleType, StringType, StructType, StructField

spark = SparkSession.builder.master("local[*]").getOrCreate()

logs_hotel_df = (
    spark.read
    .option("header", True)
    .option("sep", ",")
    .option("inferSchema", False)
    .csv("Hotel.csv")
)

logs_hotel_df = logs_hotel_df \
    .withColumn("weekend_nights", col("weekend_nights").cast(IntegerType())) \
    .withColumn("week_nights", col("week_nights").cast(IntegerType())) \
    .withColumn("avg_room_price", col("avg_room_price").cast("double")) \
    .withColumn("year", col("year").cast(IntegerType())) \
    .withColumn("month", col("month").cast(IntegerType())) \
    .withColumn("n_adults", col("n_adults").cast(IntegerType())) \
    .withColumn("n_children", col("n_children").cast(IntegerType())) \
    .withColumn("date", col("date").cast(IntegerType()))

# Календарь
start_date = "2017-01-01"
end_date = "2018-12-31"
days_diff = spark.sql(f"SELECT datediff('{end_date}', '{start_date}')") \
    .collect()[0][0]
calendar_df = spark.range(days_diff + 1) \
    .withColumn("calendar_dt",
                expr(f"date_add('{start_date}', cast(id as int))")) \
    .select("calendar_dt") \
    .orderBy("calendar_dt")

print("Календарь, первые 10 записей")
calendar_df.show(10)
calendar_df.count()

# Запрос 1.
# Вычислить среднее количество ночей, которые гости проводят в отеле
# (только для подтвержденных бронирований, с детализацией по месяцам и годам)
confirmed_bookings_df = logs_hotel_df.filter(col("status") == "Not_Canceled")

nights_df = confirmed_bookings_df.withColumn(
    "total_nights",
    col("weekend_nights") + col("week_nights")
)
avg_nights_by_month_df = nights_df.groupBy("year", "month") \
    .agg(
        avg("total_nights").alias("avg_nights_per_booking"),
        sum("total_nights").alias("total_nights"),
        count("*").alias("confirmed_bookings_count")
    ) \
    .orderBy("year", "month")


print("\nЗапрос 1")
avg_nights_by_month_df.show()


# Запрос 2.
# Определить ТОП-3 месяца по проценту отмененных броней за 2018 год

monthly_stats_2018 = logs_hotel_df \
    .filter(col("year") == 2018) \
    .groupBy("month") \
    .agg(
        count("*").alias("total_bookings"),
        sum(when(col("status") == "Canceled", 1).otherwise(0))
        .alias("canceled_count")
    )
monthly_stats_with_pct = monthly_stats_2018 \
    .withColumn(
        "canceled_percentage",
        round((col("canceled_count") / col("total_bookings")) * 100, 2)
    ) \
    .orderBy(col("canceled_percentage").desc())
top_3_months = monthly_stats_with_pct.limit(3)
result = top_3_months.select(
    col("month"),
    col("canceled_percentage")
).orderBy(col("canceled_percentage").desc())

print("Запрос 2.")
result.show(truncate=False)

# Задание 3.
# Вычислить среднее время на каждый месяц между бронированием и заездом
# в отель для подтвержденных броней.
avg_lead_time_by_month = logs_hotel_df \
    .filter(col("status") == "Not_Canceled") \
    .groupBy("year", "month") \
    .agg(
        round(avg("lead_time"), 2).alias("avg_lead_time_days"),
        count("*").alias("confirmed_bookings")
    ) \
    .orderBy("year", "month")
print("Задание 3")
avg_lead_time_by_month.show()

# Задание 4.
# Вычислить общую среднюю выручку на каждый месяц в каждом году,
# сгруппировав по всем типам бронирования для подтвержденных броней,
# и вывести это в виде сводной таблицы (PIVOT)
confirmed_revenue = logs_hotel_df \
    .filter(col("status") == "Not_Canceled") \
    .withColumn("total_nights", col("weekend_nights") + col("week_nights")) \
    .withColumn("booking_revenue", col("avg_room_price") * col("total_nights"))

monthly_avg_revenue = confirmed_revenue \
    .groupBy("year", "month") \
    .agg(
        round(avg("booking_revenue"), 2).alias("avg_monthly_revenue"),
        sum("booking_revenue").alias("total_revenue"),
        count("*").alias("bookings_count")
    ) \
    .orderBy("year", "month")

print("\nЗадание 4:")
print("Средняя выручка на каждый месяц в каждом году:")
monthly_avg_revenue.show()

pivot_table = monthly_avg_revenue \
    .groupBy("year") \
    .pivot("month") \
    .agg(round(avg("avg_monthly_revenue"), 2)) \
    .orderBy("year")


print("PIVOT TABLE:")
pivot_table.show()

# Задание 5.
# Выявить ТОП-5 постоянных гостей, которые принесли наибольшую выручку
# за все время, и показать их долю в общей выручке от постоянных гостей.
# Использовать уникальный идентификатор брони как уникальный идентификатор
# гостя, предположив, что 1 бронь = 1 гость.

revenue_per_booking = logs_hotel_df \
    .filter(col("status") == "Not_Canceled") \
    .withColumn(
        "booking_revenue",
        col("avg_room_price") * (col("weekend_nights") + col("week_nights"))
    )

repeated_guests_revenue = revenue_per_booking \
    .filter(col("repeated_guest") == 1) \
    .select("ID", "booking_revenue")

total_revenue_repeated = repeated_guests_revenue \
    .agg(sum("booking_revenue").alias("total_revenue")) \
    .collect()[0]["total_revenue"]

top_5_repeated_guests = repeated_guests_revenue \
    .orderBy(col("booking_revenue").desc()) \
    .limit(5)

top_5_with_share = top_5_repeated_guests \
    .withColumn(
        "revenue_share_percent",
        round((col("booking_revenue") / lit(total_revenue_repeated)) * 100, 4)
    ) \
    .withColumn("booking_revenue", round(col("booking_revenue"), 2))

top_5_total_revenue = top_5_with_share \
    .agg(sum("booking_revenue").alias("top_5_total")) \
    .collect()[0]["top_5_total"]

top_5_total_share = (top_5_total_revenue / total_revenue_repeated) * 100

print("\nЗадание 5.\nТОП-5 постоянных гостей:")
top_5_with_share.show(truncate=False)

print(f"Общая выручка ТОП-5: {top_5_total_revenue:.2f}")
print(f"Доля ТОП-5 в выручке от постоянных гостей: {top_5_total_share:.4f}%")

# Задание 6.
# Вывести общее количество гостей на каждый день в отеле, отсортировав
# по убыванию дат, включая дни, когда отель пустует. Также рассчитать процент
# загрузки для каждого дня, если известно,
# что общая вместимость отеля 400 человек

print("\nЗадание 6\n")
confirmed_bookings = logs_hotel_df \
    .filter(col("status") == "Not_Canceled") \
    .withColumn(
        "date_string",
        concat_ws("-", col("year"), col("month"), col("date"))
    ) \
    .withColumn("is_valid_date",
                when(
                    (col("month") == 2) &
                    (col("date") > 28) &
                    ((col("year") % 4 != 0) | ((col("year") % 100 == 0) & (col("year") % 400 != 0))),
                    False
                ).when(
                    (col("month") == 2) &
                    (col("date") > 29),
                    False
                ).when(
                    (col("month").isin([4, 6, 9, 11])) & (col("date") > 30),
                    False
                ).when(
                    (col("date") > 31) | (col("date") < 1),
                    False
                ).otherwise(True)
    ) \
    .filter(col("is_valid_date") == True) \
    .withColumn(
        "checkin_date",
        to_date(col("date_string"))
    ) \
    .filter(col("checkin_date").isNotNull()) \
    .withColumn("total_guests", col("n_adults") + col("n_children")) \
    .withColumn("total_nights", col("weekend_nights") + col("week_nights")) \
    .filter(col("total_nights") > 0)

date_sequence_df = confirmed_bookings \
    .withColumn(
        "date_sequence",
        expr("sequence(checkin_date, date_add(checkin_date, total_nights - 1), interval 1 day)")
    ) \
    .select(
        explode("date_sequence").alias("stay_date"),
        "total_guests"
    )

guests_per_day = date_sequence_df \
    .groupBy("stay_date") \
    .agg(sum("total_guests").alias("total_guests_per_day")) \
    .withColumn("total_guests_per_day", coalesce(col("total_guests_per_day"), lit(0)))

calendar_df = calendar_df.withColumn("calendar_dt", to_date(col("calendar_dt")))

all_days_guests = calendar_df \
    .join(guests_per_day, calendar_df.calendar_dt == guests_per_day.stay_date, "left") \
    .select(
        calendar_df.calendar_dt.alias("date"),
        coalesce(guests_per_day.total_guests_per_day, lit(0)).alias("total_guests")
    )

occupancy_percentage = all_days_guests \
    .withColumn(
        "occupancy_percentage",
        round((col("total_guests") / lit(400)) * 100, 2)
    )

result = occupancy_percentage \
    .orderBy(col("date").desc()) \
    .select(
        date_format("date", "yyyy-MM-dd").alias("date"),
        "total_guests",
        "occupancy_percentage"
    )

result.show()


Календарь, первые 10 записей
+-----------+
|calendar_dt|
+-----------+
| 2017-01-01|
| 2017-01-02|
| 2017-01-03|
| 2017-01-04|
| 2017-01-05|
| 2017-01-06|
| 2017-01-07|
| 2017-01-08|
| 2017-01-09|
| 2017-01-10|
+-----------+
only showing top 10 rows

Запрос 1
+----+-----+----------------------+------------+------------------------+
|year|month|avg_nights_per_booking|total_nights|confirmed_bookings_count|
+----+-----+----------------------+------------+------------------------+
|2017|    7|    3.0166666666666666|         362|                     120|
|2017|    8|    2.7189384800965017|        2254|                     829|
|2017|    9|    2.6550783912747105|        3895|                    1467|
|2017|   10|    2.7032898820608318|        4355|                    1611|
|2017|   11|    2.7241935483870967|        1689|                     620|
|2017|   12|     3.043046357615894|        2757|                     906|
|2018|    1|    2.7414141414141415|        2714|                     990|
